<a href="https://colab.research.google.com/github/vikiysr/mlprojects/blob/main/Loan%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


Loading the Dataset

In [10]:
df=pd.read_csv("/Loan_Status_train.csv")

Preview of the dataset

In [11]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


Label Encoding

In [17]:
df.replace({"Gender":{"Male":1,"Female":0},"Married":{"Yes":1,"No":0},"Education":{"Graduate":1,"Not Graduate":0
                            },"Self_Employed":{"Yes":1,"No":0},"Property_Area":{"Urban":0,"Rural":1,"Semiurban":2},
                              "Loan_Status":{"Y":1,"N":0}},inplace=True)
df.replace({"Dependents":{"0":0,"1":1,"2":2,"3+":4}},inplace=True)

Checking the null values in features

In [15]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

Description about the dataset

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    float64
 2   Married            611 non-null    float64
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    int64  
 5   Self_Employed      582 non-null    float64
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    int64  
 12  Loan_Status        614 non-null    int64  
dtypes: float64(7), int64(4), object(2)
memory usage: 62.5+ KB


Imputing the Null values of continous data with mean value

In [ ]:
df["LoanAmount"].fillna(df["LoanAmount"].mean(),inplace=True)

In [ ]:
df["Loan_Amount_Term"].fillna(df["Loan_Amount_Term"].mean(),inplace=True)

In [ ]:
df["Credit_History"].value_counts()

1.0    475
0.0     89
Name: Credit_History, dtype: int64

Imputing the null values in categorical features with mode 

In [18]:
df["Self_Employed"].fillna(df["Self_Employed"].mode()[0],inplace=True)

In [19]:
df["Credit_History"].fillna(df["Credit_History"].mode()[0],inplace=True)

Checking for null values

In [20]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History        0
Property_Area         0
Loan_Status           0
dtype: int64

Since the nullvalues in the above features are small we drop it

In [21]:
df.dropna(inplace=True)

In [22]:
df.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

Preview of Data set after cleaning the null values

In [23]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,1.0,1.0,1.0,1,0.0,4583,1508.0,128.0,360.0,1.0,1,0
2,LP001005,1.0,1.0,0.0,1,1.0,3000,0.0,66.0,360.0,1.0,0,1
3,LP001006,1.0,1.0,0.0,0,0.0,2583,2358.0,120.0,360.0,1.0,0,1
4,LP001008,1.0,0.0,0.0,1,0.0,6000,0.0,141.0,360.0,1.0,0,1
5,LP001011,1.0,1.0,2.0,1,1.0,5417,4196.0,267.0,360.0,1.0,0,1


Segregating the independent feature and dependent feature

In [24]:
X=df.drop(columns=["Loan_ID","Loan_Status"],axis=1)
y=df.Loan_Status

Standardization of feature values

In [26]:
X.std()

Gender                  0.391118
Married                 0.477657
Dependents              1.206816
Education               0.407509
Self_Employed           0.336827
ApplicantIncome      5965.429068
CoapplicantIncome    3043.448229
LoanAmount             84.052035
Loan_Amount_Term       65.555451
Credit_History          0.334827
Property_Area           0.837425
dtype: float64

In [27]:
scaler=StandardScaler()
scaler.fit(X)
standardized_df=scaler.transform(X)

In [28]:
standardized_df.std()

1.0

In [29]:
X=standardized_df

In [30]:
y=df.Loan_Status

In [31]:
X.shape

(553, 11)

Splitting the data into train and test splits

In [32]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=12)

Preparing the model

In [33]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=3)

Fitting the model

In [34]:
model.fit(X_train,y_train)

RandomForestClassifier(n_estimators=3)

Predicting the output of training data

In [35]:
y_trainpreds=model.predict(X_train)

Checking the performance of model with accuracy score

In [36]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_train,y_trainpreds)

In [37]:
print(accuracy)

0.9389140271493213


Fitting the model on test data

In [38]:
model.fit(X_test,y_test)

RandomForestClassifier(n_estimators=3)

Predicting the outcome of test data

In [40]:
y_testpreds=model.predict(X_test)

Finding the accuracy of the model on test data

In [41]:
accuracy_test=accuracy_score(y_testpreds,y_test)

In [42]:
print(accuracy_test)

0.954954954954955


Loading the validation dataset

In [44]:
df2=pd.read_csv("/Loan_Status_test.csv")

Preview of dataset

In [45]:
df2.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


Dimensions of the dataset

In [46]:
df2.shape

(367, 12)

Description about the dataset

In [47]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            367 non-null    object 
 1   Gender             356 non-null    object 
 2   Married            367 non-null    object 
 3   Dependents         357 non-null    object 
 4   Education          367 non-null    object 
 5   Self_Employed      344 non-null    object 
 6   ApplicantIncome    367 non-null    int64  
 7   CoapplicantIncome  367 non-null    int64  
 8   LoanAmount         362 non-null    float64
 9   Loan_Amount_Term   361 non-null    float64
 10  Credit_History     338 non-null    float64
 11  Property_Area      367 non-null    object 
dtypes: float64(3), int64(2), object(7)
memory usage: 34.5+ KB


Statistical Measures of the dataset

In [48]:
df2.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,367.000000,367.000000,362.000000,361.000000,338.000000
mean,4805.599455,1569.577657,136.132597,342.537396,0.825444
std,4910.685399,2334.232099,61.366652,65.156643,0.380150
min,0.000000,0.000000,28.000000,6.000000,0.000000
25%,2864.000000,0.000000,100.250000,360.000000,1.000000
50%,3786.000000,1025.000000,125.000000,360.000000,1.000000
75%,5060.000000,2430.500000,158.000000,360.000000,1.000000
max,72529.000000,24000.000000,550.000000,480.000000,1.000000


Label Encoding

In [49]:
df2.replace({"Gender":{"Male":1,"Female":0},"Married":{"Yes":1,"No":0},"Education":{"Graduate":1,"Not Graduate":0
                            },"Self_Employed":{"Yes":1,"No":0},"Property_Area":{"Urban":0,"Rural":1,"Semiurban":2}
                              },inplace=True)

In [50]:
df2.replace({"Dependents":{"0":0,"1":1,"2":2,"3+":4}},inplace=True)

Imputing the Null values with mean and mode

In [54]:
df2["LoanAmount"].fillna(df2["LoanAmount"].mean(),inplace=True)

In [55]:
df2["Loan_Amount_Term"].fillna(df2["Loan_Amount_Term"].mean(),inplace=True)

In [56]:
df2["Self_Employed"].fillna(df2["Self_Employed"].mode()[0],inplace=True)

In [57]:
df2["Credit_History"].fillna(df2["Credit_History"].mode()[0],inplace=True)

In [58]:
df2.dropna(inplace=True)

Setting the independent features to predict the output

In [59]:
X2=df2.drop(columns=["Loan_ID"],axis=1)

Predicting the output

In [62]:
y_validation=model.predict(X2)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Setting Labels for binary data

In [63]:
df2["predicted_output"]=y_validation
df2.predicted_output.replace({0:"No",1:"Yes"},inplace=True)

In [65]:
df2

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,predicted_output
0,LP001015,1.0,1,0.0,1,0.0,5720,0,110.0,360.0,1.0,0,Yes
1,LP001022,1.0,1,1.0,1,0.0,3076,1500,126.0,360.0,1.0,0,Yes
2,LP001031,1.0,1,2.0,1,0.0,5000,1800,208.0,360.0,1.0,0,Yes
3,LP001035,1.0,1,2.0,1,0.0,2340,2546,100.0,360.0,1.0,0,Yes
4,LP001051,1.0,0,0.0,0,0.0,3276,0,78.0,360.0,1.0,0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,LP002971,1.0,1,4.0,0,1.0,4009,1777,113.0,360.0,1.0,0,Yes
363,LP002975,1.0,1,0.0,1,0.0,4158,709,115.0,360.0,1.0,0,Yes
364,LP002980,1.0,0,0.0,1,0.0,3250,1993,126.0,360.0,1.0,2,Yes
365,LP002986,1.0,1,0.0,1,0.0,5000,2393,158.0,360.0,1.0,1,Yes
